In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed
import serial
import time

ser = serial.Serial("/dev/ttyUSB0", 115200)
if not ser:
    print("코디네이터가 연결되지 않았습니다")

def location(x,y):
    #print(x,y)
    plt.axis([-10,70,-10,130])
    plt.plot(x, y,'rs')
    plt.show()

wList = np.array([])
# point1 = np.array([])
# point2 = np.array([])
point1 = []
point2 = []
powerList = [np.array([]),np.array([]),np.array([]),np.array([])]
x,y = 0,0


# 신호 세기 측정 (Slider)
for i in range(4):
    wList = np.append(wList, widgets.IntSlider(max=100, description='rssi_rout'+str(i+1)))
    display(wList[i])


check = True
if check:
    check = not check
    
    start = time.time()
    while True:
        try:
            ret = ser.readline()
            if ret:
                #print(ret.decode()[:-2])        # 제일 뒤의 '\n'을 뺌

                receive = np.array([])
                receive = ret.decode()[:-2].split('#')        # 라우터이름, 신호세기

                if receive[0]=="Rout01":
                    wList[0].value = int(receive[1])
                    powerList[0] = np.append(powerList[0], np.array([int(receive[1])]))

                elif receive[0]=="Rout02":
                    wList[1].value = int(receive[1])
                    powerList[1] = np.append(powerList[1], np.array([int(receive[1])]))

                elif receive[0]=="Rout03":
                    wList[2].value = int(receive[1])
                    powerList[2] = np.append(powerList[2], np.array([int(receive[1])]))

                elif receive[0]=="Rout04":
                    wList[3].value = int(receive[1])
                    powerList[3] = np.append(powerList[3], np.array([int(receive[1])]))
                else:
                    pass
                #time.sleep(.1)

                if time.time()-start>3.5:
                    break
                    
        except(KeyboardInterrupt):
            break

    #print(powerList)
            
#     for i in range(4):
#         wList[i] = int(powerList[i].mean())
    wList = np.array([round(i.mean(),2) for i in powerList])
    #wList -= 20     # 최소거리
    print(wList)
    
    
# 좌표 구하기 - 대각선 간 비율 이용
    w = 64.1    #  공간 가로길이
    h = 129.1    #  공간 세로길이
    ratio1 = wList[1] / (wList[1]+wList[3])
    ratio2 = wList[0] / (wList[0]+wList[2])
    
    for i in range(int(w)):
        for j in range(int(h)):
            length1 = (i**2 + j**2)**(1/2)
            length3 = ((w-i)**2 + (h-j)**2)**(1/2)

            if (ratio1*length1 - (1-ratio1)*length3)**2 < 0.3 :
                point1.append((i,j))

            length0 = (i**2 + (h-j)**2)**(1/2)
            length2 = ((w-i)**2 + j**2)**(1/2)

            if (ratio2*length0 - (1-ratio2)*length2)**2 < 0.3 :
                point2.append((i,j))
    
    brk = False
    for i in point1:
        for j in point2:
            if i==j:
                print(i)
                x,y = i[0],i[1]
                brk = True
                break
        if brk:
            break

interact(location, x=x, y=y)

IntSlider(value=0, description='rssi_rout1')

IntSlider(value=0, description='rssi_rout2')

IntSlider(value=0, description='rssi_rout3')

IntSlider(value=0, description='rssi_rout4')

[26.83 32.93 34.04 29.99]
(37, 58)


interactive(children=(IntSlider(value=37, description='x', max=111, min=-37), IntSlider(value=58, description=…

<function __main__.location(x, y)>

In [19]:
wayPoint = np.array([[42.2, 13], [42.2, 33.2], [42.2, 50.9], [42.2, 68.4], [42.2, 87.6], [42.2, 117.6]])

# 현재위치(x,y)에서 경유지(way)까지의 거리측정
waylen = np.array([])
for i in range(len(wayPoint)):
    waylen = np.append(waylen,((wayPoint[i][0]-x)**2+(wayPoint[i][1]-y)**2)**(1/2))
    
print(waylen)
print(np.argmin(waylen), min(waylen))  # 가장 가까운 체크포인트, 거리
print(wayPoint[np.argmin(waylen)])     # 가장 가까운 체크포인트 좌표
print(x,y)                             # 현재 좌표

# 각도, 주행시간(=거리/속력)   # 마찰 고려. 


[45.29944812 25.33929754  8.80056816 11.62755348 30.05328601 59.82641557]
2 8.800568163476722
[42.2 50.9]
37 58


In [ ]:
# 1바퀴 회전시간 = ?
# => 각도에 해당하는 회전시간

# 속도별 측정


In [ ]:
# 회전
import time
from pop.Pilot import SerBot

omni = OmniWheel()
omni.forward([50,50,50])   # 모터를 앞쪽으로 회전 (3개모터 모두 앞으로 회전하면 제자리 회전하게됨)
time.sleep(3)
omni.backward([90,90,90])
time.sleep(3)
omni.stop()

In [30]:
import time
from pop.CAN import OmniWheel

omni = OmniWheel()
#omni.forward([15,15,15])
speed = 15
omni.backward([15 for _ in range(3)])
time.sleep(5.215)
omni.stop()

# 5초 - 405도  1 + 1/8     # 속도20 - 한바퀴 : 40/9초
# 속도15 - 한바퀴 : 5.21초

In [47]:
omni.backward([15 for _ in range(3)])
#time.sleep(5.515/2)
time.sleep(2.6075)
omni.stop()

In [198]:
from pop.Pilot import SerBot
from pop.CAN import OmniWheel
import time

bot = SerBot()
omni = OmniWheel()

def go(n):
    bot.move(-1.18, 40)
    time.sleep(n)
    bot.stop()
    time.sleep(.1)

def back(n):
    bot.move(180, 40)
    time.sleep(n)
    bot.stop()
    time.sleep(.1)
   
    
def turn(n):           # -360 ~ 360     # 양수-시계방향
    if n>0:
        omni.backward([15,15,15])
    else:
        omni.forward([15,15,15])
        n *= -1
        
    time.sleep(5.235/360*n)         # 한바퀴(5.215)
    #time.sleep(2.6075)         # 반바퀴
    omni.stop()
    time.sleep(.1)

In [200]:
go(23)

In [ ]:
from pop import xnode, time, IMU

imu = IMU()

while True:
    #xnode.transmit(xnode.ADDR_BROADCAST, "hello")
    
    packet = xnode.receive()
    
    data = ""
    for d in imu.accel(), imu.gyro():
        data += ("%.2f," * 3)%(d)
    data += ("%.2f," * 4)%(imu.quat())
    
    if packet:
        print(str(packet['payload'])[2:-1], "/", data)

    time.sleep_ms(10)
    time.sleep(.5)

In [ ]:
for i in range(3):
    acc = np.append(wList, widgets.IntSlider(max=10, description='acc'+str(i+1)))
    display(acc[i])
for i in range(3):
    gyro = np.append(wList, widgets.IntSlider(max=10, description='gyro'+str(i+1)))
    display(gyro[i])
for i in range(4):
    quat = np.append(wList, widgets.IntSlider(max=10, description='quat'+str(i+1)))
    display(quat[i])

    